In [6]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import pandas as pd
import numpy as np
import collections
import json
import math
import glob

import logging
logging.basicConfig(filename='training.log',level=logging.DEBUG)

In [7]:
training_files = glob.glob('training_data/oval/*.csv')
training_data = {}

for training_file in training_files:
    df = pd.read_csv(training_file, index_col=False)
    training_data[training_file] = df.values

TARGET_SIZE = 3
INPUT_SIZE = 22
HIDDEN_SIZE = 22

NUM_LAYERS = 1
BATCH_SIZE = 1
NUM_EPOCHS = 50
MILESTONES = [10, 40]
GAMMA = 0.1
LEARNING_RATE = 1e-2

In [10]:
def save_checkpoint(state, is_best, filepath):
    torch.save(state, filepath)
    if is_best:
        torch.save(state, 'checkpoints/best_model_full.pth.tar')
        torch.save(state['state_dict'], 'checkpoints/best_model_state.pth.tar')

In [11]:
class LSTMDriver(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, batch_size):
        super(LSTMDriver, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.batch_size = batch_size
        
        super(LSTMDriver, self).__init__()

        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, num_layers=self.num_layers)
        
        self.hidden = self.init_hidden()
        
        self.linear = nn.Linear(self.hidden_size, self.output_size)

    def init_hidden(self):
        return (autograd.Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_size)),
                    autograd.Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_size)))
    
    def forward(self, x):
        lstm_out, self.hidden = self.lstm(x.view(len(x), 1, -1), self.hidden)
        linear_out = self.linear(lstm_out)
        
        return linear_out

In [12]:
model = LSTMDriver(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, TARGET_SIZE, BATCH_SIZE)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=MILESTONES, gamma=GAMMA)
min_loss = math.inf
losses = []

for epoch in np.arange(NUM_EPOCHS):
    logging.info('Epoch [%d/%d]' %(epoch+1, NUM_EPOCHS))
    
    current_loss = 0
    is_best = False
    scheduler.step()
    for key in training_data:
        logging.info('--- parsing track: %s/%s' % (key.split('/')[1], key.split('/')[2]))

        model.hidden = model.init_hidden()
        track_sequence = training_data[key]
        
        targets = track_sequence[:, 0:3]
        inputs = track_sequence[:, 3:]
        
        targets_variable = autograd.Variable(torch.Tensor(targets))
        inputs_variable = autograd.Variable(torch.Tensor(inputs))
        
        outputs_variable = model(inputs_variable)
        
        loss = criterion(outputs_variable, targets_variable)
        
        loss.backward()
        optimizer.step()
        
        current_loss += loss.data[0]

    if current_loss < min_loss:
        print('---|--- best model found so far: %f' % current_loss)
        min_loss = current_loss
        is_best = True
    else:
        logging.info('---|--- not the best model: %f' % current_loss)
        
    save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'min_loss': min_loss,
            'optimizer' : optimizer.state_dict(),
        }, is_best, filepath='checkpoints/checkpoint.pth.tar')
    
    logging.info('-------------------------------------------------------')

---|--- best model found so far: 0.000000
